<a href="https://colab.research.google.com/github/smsarfroz/Interactive-Dialogue-chatbot/blob/main/Chatbot_made_through_python_and_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import nltk
import string
import random
import sqlite3
from textblob import TextBlob
from collections import defaultdict

In [ ]:
pip install textblob

In [ ]:
# Initializing conversation history and state
conversation_history = []
state = defaultdict(list)


In [ ]:
f=open('chatbot.txt' , 'r' , errors = 'ignore' )
f=open('dialogs.txt' , 'r' , errors = 'ignore' )
#f=open('Conversation[1].csv' , 'r' , errors = 'ignore' )
raw_doc=f.read( )
raw_doc=raw_doc.lower() #converts text to lowercase
nltk.download( 'punkt') #Us1ng the Punkt tokenizer
nltk.download( 'wordnet') #Using the WordNet dictionary
sent_tokens = nltk.sent_tokenize(raw_doc)#Converts doc to list of sentences

word_tokens = nltk.word_tokenize(raw_doc) #Converts doc to list of wc


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
sent_tokens[:2]

['hi, how are you doing?', "i'm fine."]

In [ ]:
word_tokens[:2]

['hi', ',']

In [ ]:
lemmer= nltk.stem.WordNetLemmatizer()

#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string. punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
GREET_INPUTS = ("hello", "hi","greetings", "sup","what's up", "hey",)

GREET_RESPONSES =["hi", "hey","*nods*","hi there","hello", "I am glad! You are talking to me"]
def greet(sentence):
  for word in sentence.split():
    if word.lower() in GREET_INPUTS:
      return random.choice(GREET_RESPONSES)

In [ ]:
suggestions = ["tell me about your hobbies",  "how can I contact customer support?"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def create_feedback_table():
    conn = sqlite3.connect('feedback.db')
    c = conn.cursor()

    # Create the feedback table if it doesn't exist
    c.execute('''CREATE TABLE IF NOT EXISTS feedback (
                    response text,
                    feedback text
                )''')

    # Commit the changes and close the connection
    conn.commit()
    conn.close()


In [ ]:
def collect_feedback(response):
    # ask the user to provide feedback
    print("BOT: How would you rate my response? (1-5)")
    feedback = input("User: ")

    # Store the feedback in the database
    save_feedback_to_database(response, feedback)


In [ ]:
def save_feedback_to_database(response, feedback):
    conn = sqlite3.connect('feedback.db')
    c = conn.cursor()

    # Insert the feedback into the database
    c.execute("INSERT INTO feedback (response, feedback) VALUES (?, ?)", (response, feedback))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()


In [ ]:
create_feedback_table()


In [ ]:
def response(user_response):
  robo1_response=''
  TfidfVec=TfidfVectorizer(tokenizer=LemNormalize,stop_words='english')
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals=cosine_similarity(tfidf[-1],tfidf)
  idx=vals.argsort()[0][-2]
  flat=vals.flatten()
  flat.sort( )
  req_tfidf=flat[-2]
  blob = TextBlob(user_response)
  sentiment = blob.sentiment.polarity


  if sentiment > 0:
      # Handle positive response logic
      robo1_response += "I'm glad to hear that!"
  elif sentiment < 0:
      # Handle negative response logic
      robo1_response += "I'm sorry to hear that. How can I assist you?"
  if(req_tfidf==0):
    robo1_response=robo1_response+"l am sorry! I didn't understand you. "
    return robo1_response
  else:
    robo1_response=robo1_response+sent_tokens[idx]
    return robo1_response

  # Add user response to conversation history
  conversation_history.append(user_response)

  # Update state based on conversation history
  state["user"].append(user_response)
  state["bot"].append(robo1_response)

  dialogue_response = generate_dialogue_response(user_response)

In [ ]:
def generate_dialogue_response(user_response):
    # Implement your dialogue manager logic here
    if len(conversation_history) > 1:
        # Generate a response based on the previous user input and bot response
        previous_user_input = conversation_history[-2]
        previous_bot_response = state["bot"][-1]

        # Customize your dialogue manager logic based on the previous interaction
        if previous_user_input == "Tell me a joke" and "joke" in previous_bot_response:
            return "Sure! Why don't scientists trust atoms? Because they make up everything!"
        elif previous_user_input == "What is your favorite movie?" and "movie" in previous_bot_response:
            return "I am a bot, so I don't have favorite movies. But I can recommend some popular ones!"

    # If no specific context matches, generate a generic response
    return "I'm sorry, I didn't understand. Can you please rephrase or provide more information?"


In [ ]:
flag=True
print("BOT: MY name is Chatnlt. Let's have a talk! and,if you want to quit any time ,just type Bye")
while(flag==True):
  user_response=input()
  user_response=user_response.lower()
  if(user_response!='bye'):
      if(user_response=='thanks' or user_response=='thank you'):
        flag=False
        print("BOT: You are welcome.. ")
      else:
          if(greet(user_response)!=None):
            print("BOT: "+greet(user_response))
          else:
            sent_tokens.append(user_response)
            word_tokens=word_tokens+nltk.word_tokenize(user_response)
            final_words=list(set(word_tokens))
            print("BOT: ",end="")

            dialogue_response = generate_dialogue_response(user_response)

            chatbot_response = response(user_response)
            print(chatbot_response)
            collect_feedback(chatbot_response)

            sent_tokens.remove(user_response)

  else:
     flag=False
     print("BOT: Goodbye! Take care")

BOT: MY name is Chatnlt. Let's have a talk! and,if you want to quit any time ,just type Bye
BOT: hey
BOT: 

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


i don't like that man.
BOT: How would you rate my response? (1-5)
BOT: I'm glad to hear that!i liked it.
BOT: How would you rate my response? (1-5)
